# Fundindo DataFrames (Merge)

Nesta sessão vamos vamos aprender a juntar dataFrames, tanto horizontalmente como concatenando verticalmente. 

Para entender isso vamos conceituar o processo como algo semelhante a um diagrama de veins

Diagramas de veins separam os valores em categorias, e permitindo enxergar valores que representam nenhum, uma ou mais categorias.


Quando trazemos isso ao pandas, podemos pensar que temos duas populações, com indices separados. Quando quisermos juntar esses dataframes, temos algumas escolhas a tomar.

Para exemplificar. Vamos supor dataframes de alunos e de funcionários de alguma faculdade. Se quisermos uma lista de todos. Em linguagem de dados seria um *full outer join*. E na teoria de probabilidade seria uma união. (Todos os circulos do diagrama).

É possível que queiramos apenas aqueles que são funcionários e estudantes. *Inner Join* (A intersecção do diagrama).

In [1]:
#Vamos ver como isso funciona com pandas na prática.

import pandas as pd

#Vamos criar dois dataframes

staff = pd.DataFrame([{'Nome' : 'Kelly', 'Cargo' : 'Diretor'},
                      {'Nome' : 'Sally', 'Cargo' : 'Monitor'},
                      {'Nome' : 'James', 'Cargo' : 'Professor'}])

staff = staff.set_index('Nome')

student = pd.DataFrame([{'Nome' : 'James', 'Curso' : 'Empreendedorismo'},
                        {'Nome' : 'Mike' , 'Curso' : 'Direito'},
                        {'Nome' : 'Sally' , 'Curso' : 'Engenharia'}])

student = student.set_index('Nome')

print(student.head())

print(staff.head())

#Importante que o indice do dataframe seja aquilo que queiramos juntar


                  Curso
Nome                   
James  Empreendedorismo
Mike            Direito
Sally        Engenharia
           Cargo
Nome            
Kelly    Diretor
Sally    Monitor
James  Professor


In [2]:
#Se quisermos a união destes dataFrames, chamamos o função 'merge()' passando os dois dataframes, e dizendo que queremos outerjoin. Queremos usar os indices
# da direita e da esquerda e juntar as colunas

pd.merge(staff, student, how='outer', left_index=True, right_index=True)

,Cargo,Curso
Nome,,
James,Professor,Empreendedorismo
Kelly,Diretor,NaN
Mike,NaN,Direito
Sally,Monitor,Engenharia


In [3]:
#A união dos dois dataframes retornou todos os valores nos dataframes. Se quisermos pegar uma intersecção, ou seja, apenas as pessoas que são estudantes e funcionaios
# Devemos usar o atributo 'inner'.
pd.merge(staff,student, how='inner', left_index = True, right_index = True)


,Cargo,Curso
Nome,,
Sally,Monitor,Engenharia
James,Professor,Empreendedorismo


In [4]:
#Agora, dois outros casos interessantes, ambos que podem ser chamados de 'definir adição'.  Aprimeira é se quisermos uma lista de todos os funcionarios,
#independente se são estudantes ou não. Mas se eles forem, gostariamos de vizualizar os detalhes. Podemos fazer isso com um 'left join', é importante notar
#a ordem dos dataframes na função. o da esquerda é o da esquerda, o segunda da direita.

pd.merge(staff, student, how='left', left_index=True, right_index=True)



,Cargo,Curso
Nome,,
Kelly,Diretor,NaN
Sally,Monitor,Engenharia
James,Professor,Empreendedorismo


In [5]:
#Obviamente, podemos tentar fazer o inverso, vizualizar todos que são estudantes, e suas funções caso sejam funcionarios. 
pd.merge(staff, student, how ='right', left_index=True, right_index=True)

,Cargo,Curso
Nome,,
James,Professor,Empreendedorismo
Mike,NaN,Direito
Sally,Monitor,Engenharia


In [6]:
#Nós podemos fazer isso de outra forma também. O método merge tem alguns outros interessantes parametros. Primeiro, você não precisa usar indices
#Você também pode usar as colunas, por exemplo: Vamos usar o parametro chamado 'on', e podemos assimila-lo aos dataframes e juntando como colunas:

#primeiro, vamos remover os indices dos dois dataframes:
staff = staff.reset_index()
student = student.reset_index()

#Agora, vamos junta-los usanndo o parametro 'on':
pd.merge(staff, student, how='right', on='Nome')

,Nome,Cargo,Curso
0,James,Professor,Empreendedorismo
1,Mike,NaN,Direito
2,Sally,Monitor,Engenharia


In [7]:
#Então, o que acontece se tivermos conflitos entre os os DataFrames? Vamos dar uma olhada criando novos dataframes e adicionando o Local.

staff = pd.DataFrame([{'Nome': 'Kelly', 'Cargo': 'Diretor de filme', 'Local' : 'Rua Estadual'},
                      {'Nome': 'Sally', 'Cargo': 'Monitor', 'Local': 'Avenida Washington'},
                      {'Nome': 'James', 'Cargo': 'Professor', 'Local': 'Avenida Washington'}])

student = pd.DataFrame([{'Nome': 'James', 'Curso': 'Empreendedorismo', 'Local': '1024 Avenida Paulista'},
                        {'Nome' : 'Mike', 'Curso' : 'Direito', 'Local': 'Republica #22'},
                        {'Nome' : 'Sally', 'Curso' : 'Engenharia', 'Local': 'Faria Lima'}])

#No dataframe dos funcionários, tem o local do escritório. No dataframe dos estudantes, o local representa sua residencia.

#O método 'merge' mantem essa informação, mas acrescenta um _x/_y para ajudar a diferenciar de qual dataframe veio a informaçãp
# O _x é sempre o dataframe da esquerda e o _y da direita.

#Agora se quisermos a informação de todos os funcionários independente se são estudantes ou não. Mas se forem, exibe os detalhes.
pd.merge(staff, student, how='left', on='Nome')

,Nome,Cargo,Local_x,Curso,Local_y
0,Kelly,Diretor de filme,Rua Estadual,NaN,NaN
1,Sally,Monitor,Avenida Washington,Engenharia,Faria Lima
2,James,Professor,Avenida Washington,Empreendedorismo,1024 Avenida Paulista


In [8]:
 #Como dito antes, e reiterado pela saida, podemos ver que existe a coluna, local_x e local_y, onde o x representa o local do staff dataframe (left),
 #e o y o dataframe dos estudantes(right).

 #Antes de deixar o assunto da fusão de dataFrames (merging), Vamos falar sobre multi-indices e colunas.
 #Seria bem comum, que o nome de um funcionário e um estudante fossem os mesmos, mas não o sobrenome. Neste caso, usamos uma lista de multiplas colunas
 #que devem ser passadas como as chaves a serem juntadas ('Join keys'), de ambos os dataframes como parametro. Veja um exemplo:

staff = pd.DataFrame([{'Nome' : 'Kelly', 'Sobrenome': 'Matos', 'Cargo' : 'Diretora'},
                      {'Nome' : 'Sally', 'Sobrenome' : 'Borges', 'Cargo' : 'Monitora'},
                      {'Nome' : 'James', 'Sobrenome' : 'Whinderson', 'Cargo'  : 'Professor'}])

student = pd.DataFrame([{'Nome' : 'James', 'Sobrenome' : 'Havan', 'Curso' : 'Empreendedorismo'},
                        {'Nome' : 'Michel', 'Sobrenome' : 'Santos', 'Curso' : 'Direito'},
                        {'Nome' : 'Sally', 'Sobrenome' : 'Borges' , 'Curso' : 'Engenharia'}])

#Como você ve aqui, Temos dois James diferentes, logo suas duas chaves não vão bater mutuamente. Então se fizermos um 'inner join' (intersecção),
#teremos apenas aqueles que são funcionarios e alunos.
pd.merge(staff, student, how='inner', on = ['Nome','Sobrenome'])
  


,Nome,Sobrenome,Cargo,Curso
0,Sally,Borges,Monitora,Engenharia


In [9]:
#juntar dataframes é uma tarefa bem comum e você precisa saber como selecionar os dados de diferentes fontes, limpa-los, então concatena-los
#(junta-los verticalmente) e enfim analisa-los.

#Podemos pensar 'merging' como juntar horizontalmente valores similares em uma coluna encontrada em dois dataframes e concatenar como juntar verticalmnte
#Significando como colocassemos um dataframe no topo ou fim um do outro.

#Vamos usar um exemplo, você tem um dataset que contem algumas informações através de alguns anos, e cada ano é um arquivo CSV de mesmo número de colunas
#O que acontece se quisermos puxar todos os dados, de todos os anos arquivados, juntos? Podemos concatena-los.

In [10]:
#Vamos dar uma olhada nos dados do 'US Departament of Education College Scorecard'. Ele contem cada universidade dos USA e dados gerais sobre seus alunos
#Os dados estão guardados em diferentes CSV's para diferentes anos. Vamos supor que queremos os arquivos entre 2011 e 2013 nós primeiro criamos três dataframes
#(um para cada ano),  e como os dados estão meio bagunçados, vamos se esquivar de alguns erros que provavelmente acontecerá no ambiente de execução. 
#Para isso vamos usar um comando de celula magica chamda %%capture

In [11]:
%%capture

df_2011 = pd.read_csv('MERGED2011_12_PP.csv', error_bad_lines=False)
df_2012 = pd.read_csv('MERGED2012_13_PP.csv', error_bad_lines=False)
df_2013 = pd.read_csv('MERGED2013_14_PP.csv', error_bad_lines=False)

FileNotFoundError: ignored

In [12]:
#Vamos dar uma olhada:
df_2011.head(5)

NameError: ignored

In [ ]:
#Vemos que temos mais de 2000 colunas, vamos ver o tamanho de cada dataframe
print(len(df_2011))
print(len(df_2012))
print(len(df_2013))

In [ ]:
#Vamos colocar todos os dataframes em uma lista a cja,a essa lista na função 'concat()'

frames = [df_2011, df_2012, df_2013]
pd.concat(frames)



In [ ]:
len(df_2011) + len(df_2012) + len(df_2013)

In [ ]:
#Vemos que a quantidade de dados do dataframe gerado pela concatenação é identica a soma dos 3 dataframes. Mas agora
#temos um problema pois não sabemos de qual ano(dataframe) corresponde cada dado.

#Podemos resolver com um parametro da função 'concat()', que o parametro 'keys' onde podemos colocar um nivel extra
#de indices. Nós passamos uma lista de chaves que queremos corresponder cada dataframe, veja:

pd.concat(frames, keys=['2011','2012','2013'])


In [ ]:
#Agora temos um multi indice de cada ano. A concatenação também possui um inner e outer method. Se estivermos concatenando 
#dois dataframes que não possui colunas identicas, e escolher um método 'outer', algumas celulas vão ser NaN. Se escolher
#um método 'inner' então algumas informações NaN vão ser removidas. Você pode pensar nisso analogamente ao 'left'/'right'
#join, da função 'merge()'

#Pandas Idiomaticas


É comum, assim como na matemática, que digam que existem mais de uma solução para um problema. Ou na computação, várias maneiras de escrever um código que faz a mesma coisa. Mas algumas maneiras, ou soluções são mais apropriadas que outras. As melhores soluções são tomadas como Idiomáticas e geralmente são as mais usadas em exemplos, na internet/StackOverFlow.

Um conjunto de sub-linguagem dentro do python, no caso Pandas, tem seus próprio conjunto idiomático. Vamos ver alguns agora, como vetorizar sempre que possivel, e não uar laços(loopings) se não precisar. Os desenvolvedores e usuários usam o termo **Pandorable**. Vamos lá:

In [ ]:
import pandas as pd
import numpy as np
import timeit

In [ ]:
#E vamos novamente usar o dataset do census.
df = pd.read_csv('census.csv')
df1 = pd.read_csv('census.csv')
df.head()

In [ ]:
#De primeira, vamos falar do método encadeamento 'chaining'. A ideia geral é que todo método do objeto retorne a referencia deste objeto.
# A beleza disso é condensar diversas operações no dataframe, em apenas uma linha ou apenas um argumento.

#vamos ver yma forma pandoravel de encadeamento:

(df.where(df['SUMLEV'] ==50)
    .dropna()
    .set_index(['STNAME','CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))



In [ ]:
#Primeiramente, vamos usar a função 'where()' no dataframe e passa-lo em uma mascara booleana que é apenas True para 
#as linhas onde SUMLEV é igual a 50. Isso indica na fonte do nosso dados foi resumido ao nível dos condados. Com
#o resultado do 'where()' avaliado, nós removeremos (drop()) os valores em branco. Apenas lembrando que o where não
#remove os valores nulos por padrão. Então colocamos um indice no dataframe que queriamos no resultado e por fim 
#renomeamos uma coluna. Quero que perceba, que ao invés de escrever o comando várias vezes, ou em apenas uma linha,
#como poderia ser feito, iniciamos com um paretensis para dizer ao python "Eu vou spammar esse comando em diversas linhas
#para tornar mais legivel."

In [ ]:
#Vamos ver um exemplo de uma maneira não 'pandoravel' de escrever isso. Lembrando que não tem nada de errado com esse
#código no sentido funcional. Pode até ser mais compreensivel, apenas não é uma maneira 'pandoravel' de escreve-lo.

#Primeiro criamos um novo dataframe do original e chamamos de df1
df = df1[df1['SUMLEV'] == 50] #Desta forma, os NaNs já são dropados automaticamente
#Atualizamos o dataframe para haver novos indices, usaremos inplace = True, para realizar a operação no data frame mesmo
df.set_index(['STNAME','CTYNAME'], inplace= True)
#Vamos alterar o nome da coluna agora.
df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})
df.head()

In [ ]:
#A chave idiomática do python é entender quando isso não te ajuda, para isso, vamos criar uma função e verificar qual
#método é mais rapido.
#Para isso vamos criar funções e ver o seu tempo de execução, então:

def first_approach():
  global df
  return (df.where(df['SUMLEV'] ==50)
          .dropna()
          .set_index(['STNAME','CTYNAME'])
          .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'}))
  
df = pd.read_csv('census.csv')
timeit.timeit(first_approach, number = 10)

In [ ]:
#agora testando a segunda abordagem, como pode ter percebido. Nós usamos uma variavel global na primeira abordagem
#Contudo, alterar uma variavel global dentro de uma função, modificará mesmo no escopo global, e não queremos isso no momento
#então, então na hora de aplicar a mascara escolhendo os valores 'SUMLEV' == 50, vamos criar um novo dataframe, então:

def second_approach():
  global df
  new_df = df.where(df['SUMLEV'] == 50)
  new_df.set_index(['STNAME', 'CTYNAME'])
  new_df.rename(columns={'ESTIMATESBASE2010' : 'Estimates Base 2010'})
  return new_df

df = pd.read_csv('census.csv')
timeit.timeit(second_approach, number = 10)

In [ ]:
#Como acabamos de ver, o segundo método é mais rápido, e esse é um classico exemplo de legibilidade vs tempo
#Você verá muitos exemplos de encadeamentos pelos forums de computação e é importante que você consiga entender, mas tenha
#em mente que trás alguns problemas de performance essa idiomática.

In [ ]:
#Outra função idiomática do panda. O python possui a incrivel função map. Que é uma espécie de base para programação funcional
#Quando usamos map no python, você passa uma função que queremos aplicar, e um iteravel, como uma lista, da qual queremos
#que a função seja aplicada. O resultado é que o a função é aplicada para cada item, e retorna uma lista com todos os resultados.

#Pandas possui uma função semelhante chamada 'applymap'. Nesta função, você fornece uma função que quer operar em cada célula
#do dataframe, mas é pouco usado. Ao invés, nos vemos querendo aplicar a função através de cada linha do dataframe.
#E o pandas tem uma função para isso chamada 'apply',vamos ver um exemplo:


In [ ]:
#Ainda usando o census dataframe, temos 5 colunas que são relacionadas a estimativa populacional do ano. É bem razoavel querer
#criar novas colunas para os valores maximos e minimos, da para fazer isso de maneira fácil usando apply  e as funções
#de maximos e minimos do numpy.

#Primeiro precisamos escrever uma função que pega uma linha de interesse, encontre os valores de max e min e retorne uma nova
#linha de dados. Nós podemos criar alguns pequenos slices de uma linha por projetar as colunas da população.

#Então usamos as funções do numpy de max e min, e então criar uma nova série com o rotulo dos valores que representem os novos
#valores que queremos aplicar. 

def min_max(row):
  data = row[['POPESTIMATE2010',
              'POPESTIMATE2011',
              'POPESTIMATE2012',
              'POPESTIMATE2013',
              'POPESTIMATE2014',
              'POPESTIMATE2015']]
  return pd.Series({'min': np.min(data), 'max' : np.max(data)})

In [ ]:
#Então, apenas precisamos chamar o apply no dataframe

#apply pega a função e o eixo que queremos aplicar os parametros. Agora, nós temos que ser cuidadosos. Nós temos falado 
#Sobre o eixo 0 ser as linhas do dataframe. Mas neste parametro, é realmente o parametro do indice a usar. Então, para usar
#Apply por todas as linhas, aplicando em todas as colunas, você passar o eixo sendo igual as colunas.

df.apply(min_max, axis= 'columns').head(10)

In [ ]:
#Claro que não precisamos nos limitar a criar um novo objeto (dataframe), se estivermos fazendo uma limpeza nos dados,
#poderemos querer adicionar esses novos dados ao dataframe, neste caso, apenas pega os valores das linhas e adiciona a uma
#nova coluna que indicam max e min. Isso faz parte do trabalho regular de trazer os dados e construir um resumo descritivo 
#da estatistica. Isso é muito usado quando fundindo dataFrames. 

In [ ]:
#Vamos criar um exemplo revisando a nossa função de max e min anterior. Ao invés de retornar uma série separada, vamos criar
#novas colunas no dataframe para armazenar esses dados. 

def min_max(row):
  data = row[['POPESTIMATE2010',
              'POPESTIMATE2011',
              'POPESTIMATE2012',
              'POPESTIMATE2013',
              'POPESTIMATE2014',
              'POPESTIMATE2015']]
  row['max'] = np.max(data)
  row['min'] = np.min(data)
  return row

#Agora apenas usamos apply através do dataframe.

df.apply(min_max, axis='columns')


In [ ]:
#Apply é uma funçao bem util de nosso arsenal. A razão pelo qual foi introduzido é porque raramente vemos ela sendo usada 
# Com funções de larga definição. Como fizemos. Ao invés, vemos tipicamente isso sendo usado com lambdas.

# Aqui, podemos imaginar como encadear várias chamadas de apply com uma função lambda juntos para criar uma legivel e sucinta
# Manipulação, veremosum exemplo de usar lambda e apply para encontrar os max e min.

rows = ['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013', 'POPESTIMATE2014','POPESTIMATE2015']

#Agora aplicaremos isso através dataframe com a função lambda.

df.apply(lambda x: np.max(x[rows]), axis= 1).head() # axis = 1 é sinomino de axis = columns

In [ ]:
#Caso não lembre o que é uma função lambda. Uma função lambda é apenas uma função sem nome que pega parametros e retornam
#apenas um valor, neste caso, o maximo através de todas as colunas associadas as linhas 'x'.

#A beleza do apply é que nos da uma maior flexibilidade em fazer a manipulação que quisermos. Como a função que passamos pode
#ser customizada. Vamos supor que queremos separar os estados em categorias: Centroeste, nordeste, sul e oeste.

def get_state(x):
  northeast = ['Connecticut', 'Maine','Massachusetts','New Hampshire','Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
  midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa','Kansas','Minnesota','Missouri','Nebraska','North Dakota','South Dakota']
  south = ['Delaware','Florida','Georgia','Maryland','North Carolina','South Carolina','Virginia','District of Columbia','West Virginia',
           'Alabama','Kentucky','Mississipi','Tennessee','Arkansas','Lousiana','Oklahoma','Texas']
  west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah','Wyoming','Alaska','California','Hawaii','Oregon','Washington']

  if x in northeast:
    return 'Northeast'
  elif x in midwest:
    return 'Midwest'
  elif x in south:
    return 'South'
  elif x in west:
    return 'West'

In [ ]:
#Agora que customizada a função, vamos criar uma nova coluna chamada 'região'. Que mostrará a região de um estado. Podemos usar
# a função, o comando 'apply' para isso. 

df['região'] = df['STNAME'].apply(lambda x: get_state(x))
df[['STNAME','região']].tail(10)

#Agrupar

As vezes queremos selecionar os dados com base em grupos e compreender os dados agrupados categóricamente. Vimos que o pandas nos permite iterar sobre todas as linhas de um dataframe. Isso é genericament um processo muito lento. Felizmente, Pandas possui uma função 'groupby()' para acelerar tal tarefa. A idéia por trás dessa função é que ela pega um dataframe, divide em pedaços baseado em alguns valores chaves, e aplica a computação sobre esses valores. E então junta novamente os pedaços resultantes em um novo dataframe. Em pandas isso e chamado como 'Dividir - Aplicar - Combinar' padrões.

##Splitting (Dividindo)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Vamos usar novamente o census.csv, então

df = pd.read_csv('census.csv')
#E vamos excluir o 'State level summarizations, que contem sum level de 40
df[df['SUMLEV'] == 50]
df.head()

In [ ]:
#No primeiro exemplo, vamos pegar uma lista contendo todos os estados, então vamos iterar pelos estados e para cada estado
#vamos reduzir o dataframe e calcular e media .

#Vamos fazer essa tarefa 3 vezes e marcar o tempo, para isso, vamos usar a celula mágica com a função %%timeit

In [ ]:
%%timeit -n 3

for state in df['STNAME'].unique():
  #Vamos calcular a média usando numpy
  avg = np.average(df.where(df['STNAME'] == state).dropna()['CENSUS2010POP'])
  # E vamos imprimir isso na tela

  print('Counties in state ' + state + 'have an average population of ' + str(avg))

In [ ]:
#Vamos ver novamente, agora com outro método. Vamos começar por dizer ao pandas que estamos interessados em agrupar pelo 
#nome do estado, isso é o 'split'.

%%timeit -n 3

for group, frame in df.groupby('STNAME'):
  #Você deve ter percebido que colocamos duas variaveis na iteração. Isso porque a função groupby() retorna uma tupla, onde
  #o primeiro valor é a chave, que nesse caso, é o nome do estado, e o segundo é projeção do dataframe encontrado para esse grupo

  #Agora incluiremos a lógica do 'apply', que irá calcular a média, então:
  avg = np.average(frame['CENSUS2010POP'])
  # E imprimir os resultados:
  print('Counties in state ' + group + ' have an average population of ' +str(avg))

In [ ]:
#Como podemos concluir, existe uma grande diferença na velocidade.

#Em 99% do tempo, você irá usar 'groupby()' em uma ou mais colunas. Mas você também pode criar uma função para agrupar
#e usa-la para segmentar seus dados.

#ISso é um exemplo fabricado,mas vamos dizer que temos um montante de trabalho, com muita coisa para processar, e você
#quer trabalhar em terços de estados ao mesmo tempo. Nós poderiamos criar alguma função que retorna um número entre zero e dois
#Baseado na inicial de cada estado. Então podemos usar o 'groupby()' para usar essa função e dividir o dataframe. É importante
#perceber que para realizar isso devemos colocar a coluna que queremos agrupar como indice do nosso dataframe.

df = df.set_index('STNAME')

def batch(item):
  if item[0] < 'M':
    return 0
  if item[0] < 'Q':
    return 1
  return 2

#O dataframe deve ser agrupado de acordo com o numero do lote (batch), E vamos iterar por cara grupo de lote
for group, frame in df.groupby(batch):
  print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')

In [ ]:
#Repare que desta vez não passamos o nome de uma coluna para agrupar. Como alternativa, Colocamos o index do dataframe para
#ser STNAME, e se nenhuma coluna for passada no identificador do 'groupby()', a função irá automaticamente usar o index

In [ ]:
#Vamos olhar mais um exemplo de como podemos agrupar dados. Desta vez, vamos usar o dataset fornecido pelo airbnb.
#Neste dataset temos duas colunas de interesse, 'cancellation_policy' e 'review_scores_value.
df = pd.read_csv('listings.csv')
df.head()

In [ ]:
#Então, como podemos agrupar os dados com base em duas colunas? Uma primeira abordagem, seria promover o dataframe
#em um nivel multi indices, e então apenas chamar a função 'groupby()'
df = df.set_index(['cancellation_policy','review_scores_value'])
 
 #Quando temos um multi indice nós precisamos passar em levels (ou camadas de indice) que estamos interessados em agrupar
for group, frame in df.groupby(level = (0)):
   print(group)

In [ ]:
#Isso parece funcionar. Mas se quisermos agrupar pelas politicas de cancelamento e revisão das notas, mas separando todos
#com nota dez dos que possuem nota inferior a 10. Neste caso, podemos usar uma função para gerir os agrupamentos.

def grouping_fun(item):
  #Observe que os indices devem estar organizados em primeiro nivel a "cancellation policy" e o segundo nivel
  #score_review_value

  if item[1] == 10:
    return(item[0], '10.0')
  else:
    return(item[0], 'not 10.0')

for group, frame in df.groupby(by = grouping_fun):
  print(group)

In [ ]:
df.head()

In [ ]:
#Até este ponto, aplicamos simples processamentos nos nossos dados,após separa-lo, apenas verificando saídas para demonstrar
#como o 'splitting' funciona. Os desenvolvedores do pandas tem tres largas categorias de processamento de dados, para acontecer
#durante o passo de 'apply()'. Agregação dos grupos de dados, Transformação dos grupos de dados e a Filtragem dos grupos de dados.

##Aggregation (Agregamento)


In [ ]:
#O passa mais direto do apply é o agregamento de dados, e usa o método 'agg()' no objeto 'groupby()'. Até agora, nós apenas 
#iteramos pelo os objetos dos agrupamentos, desempacotando em rotulos (nomes dos grupos) e o dataframe. Mas com 'agg()' podemos
#passar um dicionário de colunas que nós estamos interessados em agregar juntamente com a função que estamos buscando 'apply'
#a agregação.
import pandas as pd
import numpy as np
df = pd.read_csv('listings.csv')

#Agora, vamos usar 'cancellation policys' e encontrar a média das notas de avaiação:

df.groupby('cancellation_policy').agg({'review_scores_value' : np.average})
#Nas versões mais recentes do pandas, .agg() em um groupby objeto foi descontinuado, isso significa
#que devemos passar em funções customizadas para obter o efeito similar.

In [ ]:
#Bem, isso não funcionou muito bem. Apenas um monte de valores (Not a Number NaN). O problema está na função,
#Que mandamos a ser agregada, np.average não ignora os valores NaN, Contudo tem uma função que podemos usar.
df.groupby('cancellation_policy').agg({'review_scores_value': np.nanmean})

In [ ]:
#Nós podemos extender esse dicionario para agregar multiplas funções ou multiplas colunas.
df.groupby('cancellation_policy').agg({'review_scores_value' : (np.nanmean,np.nanstd),
                                       'reviews_per_month' : np.nanmean})

In [ ]:
#Vamos tomar um momento para ter certeza que entendeu a ultima celula, desde que isso é bem complexo. Primeiro, estamos
#fazendo um agrupamento do dataframe com base na coluna da politica de cancelamento. Isso cria um objeto 'groupby()'.
#Então invocamos a função 'agg()' neste objeto. A função agg vai aplicar uma ou mais funções que especificarmos ao grupo de dataframes
#e retorna uma unica linha por grupo/dataframe. Quando nós chamamos essa função, enviamos duas entradas e dicionários
#Cada uma indicando nas chaves, quais colunas queremos aplicar, e então a função a ser aplicada. Na primeira chave, passamos
# uma tupla, com duas funções. Perceba que cada função retornou em cada linha um valor. Perceba que não chamamos as funções,
#como estamos acostumados, com parenteses. 

##Transformação (Transformation)

In [ ]:
#Transforamação é diferente de agragação , enquanto agg() retorna uma linha de valor para cada grupo, a transformação retorna
#um objeto que possui o mesmo tamanho do grupo. Essencialmente, isso reproduzirá a função por todo o grupo e retornará 
#um novo dataframe.Isso faz a combinação de dados mais fácil.

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('listings.csv')

In [ ]:
#Por exemplo, suponha que queremos incluir os valores da média de um dado grupo, em base na 'cancellation policy' mas preservando o formato
#do dataframe, então, o que poderiamos gerar a diferença uma observação individual e a soma.

#Primeiro, vamos selecionar as colunas que nos interessa.
cols = ['cancellation_policy','review_scores_value']
#Agora, vamos transforma-lo.

transform_df = df[cols].groupby('cancellation_policy').transform(np.nanmean)
transform_df.head(10)

In [ ]:
#Então podemos ver que o indice aqui é atualmente o mesmo do dataframe original. Então vamos uni-lo ao dataframe. Antes
#disso, vamos renomear a coluna que não se trata mais dos valores das avaliações.
transform_df.rename({'review_scores_value':'review_scores_mean'}, axis= 1, inplace= True)
df = df.merge(transform_df, left_index= True, right_index=True)
df.head()

In [ ]:
df['review_scores_mean']

In [ ]:
#Agora podemos criar a diferença de uma dada linha, com o valor da média do grupo.
df['mean_diff'] = np.absolute(df['review_scores_value']-df['review_scores_mean'])
df['mean_diff'].head()

##Filtrando (Filtering)

In [ ]:
#Um objeto resultado da função 'groupby()' (agrupamento), tem suporte para filtrar grupos. Isso é interessante quando
# queremos agrupar por alguma caracteristica, e então fazer algumas transformações aos grupos e remover alguns grupos 
#quando estamos limpando os dados. A função 'filter()' pega em uma função que é aplicada a cada grupo e então retorna
#Um valor booleano, e dependendo do resultado, o grupo é incluido nos resultados ou não.

In [ ]:
#Por exemplo, se quisermos apenas os grupos que possuem uma média maior acima de 9 incluidas em nosso resultado
df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value']) > 9.2 )

In [ ]:
#Note que os resutados continuam indexados, mas os valores do grupo com média abaixo de 9.2 não foram copiadas.

## Aplicando (Applying)

In [ ]:
#Essa é de longe a operação mais comum invocada quando está se agrupando é a função 'apply()'. Ela nos permite, 
#aplicar uma função arbitrária para cada grupo, e costurar os resultados de volta para cada 'apply()' em um unico dataframe
#onde os indices se mantem preservados.

#Vamos ver um exemplo, ainda nos mesmos dados.

df = pd.read_csv('listings.csv')
# Vamos selecionar as colunas que nos interessam.

df = df[['cancellation_policy','review_scores_value']]
df.head()

In [ ]:
#No exemplo anterior, procuramos encontrar a média das avaliações de cada grupo e então calculamos os desvios padrôes de
#cada grupo. Isso foi um processo de dois passos, primeiro transformamos o objeto do 'groupby', e então transmitimos para 
#criar uma nova coluna. COM 'apply()', nós podemos reduzir essa lógica.

def calc_mean(group):
  #O grupo é um dataframe seja lá o parametro do nosso agrupamento, no nosso caso, é a 'cancellation policy', podemos trata-lo
  #como um dataframe completo.

  avg = np.mean(group['review_scores_value'])
  #Agora transmitiremos a formula para criar uma nova coluna.
  group['review_scores_mean'] = np.abs(avg - group['review_scores_value'])
  return group

#Agora apenas aplicaremos isto aos grupos
df.groupby('cancellation_policy').apply(calc_mean).head()

In [ ]:
# Usar apply pode ser mais lento do que usar algumas funções especializadas, especialmente o 'agg()', Mas se o dataframe não é
# gigante, isso é uma firme abordagem geral.

#Scales (Escala?, Escamas?)

É um método usado para normalizar o alcance de variaveis ou caracteristicas independentes dos nossos dados. Em processamento de dados isso também é conhecido como normalização de dados e geralmente é feito durante a etapa de processamento dos dados.


##Escalas de proporção(Ratio Scales):


*   As unidades são igualmente espaçadas
*   As operações matemáticas fundamentais são válidas (+-/*)
*   Ex: Peso, Altura.




## Escala de intervalo

*  As unidades são igualmente espaçadas, mas não existe um zero real (Portanto as operações '*/' não são válidas).

* ex: Temperatura, direção de uma bussola.

Isso pois o valor Zero tem um significado. No caso da temperatura, o 0 não significa ausencia de calor. Na bussola, 0º não é uma ausencia de direção, mas uma direção em sí.

##Escala Ordinária

*  A ordem das unidades são importantes. Mas não são igualmente espaçadas.

*  Um exemplo é o método avaliativo americano, como A+, A.

##Escala Nominal

*  Os dados são categoricos, mas as categorias não tem relação uma com a outra

*  Um exemplo seria times de um esporte.

Dado a importância dos tipos de dados para Machine Learning, pandas, tem um número interessante de funções para lidar e converter entre escalas de medidas. Vamos começar com os dados nominais, que em pandas são chamados de dados **categóricos**.
Pandas atualmente tem um tipo embutido para dados categóricos, e podemos escolher uma coluna para simplificar usando o método 'astype'. Astype tenta mudar o dado por baixo o tipo de dado, neste caso, para dados categóricos, e mais adiante muda-lo para dados ordinários.

In [ ]:
import pandas as pd
#Para exemplificar, vamos criar um um dataframe das letras de notas do sistema avaliativo americano. Vamos criar os indices
#E fazer um pouco de julgamento humano de quão bem é o desempenho do aluno.

df = pd.DataFrame(['A+','A','A-','B+','B','B-','C+','C','C-','D+','D'],
                  index = ['excelente','excelente','excelente','bom','bom','bom','regular','regular','regular','ruim','ruim'],
                  columns = ['Notas'])
df


In [ ]:
#Vamos checar os tipos dos dados, retornará como obejetos, uma vez que passamos apenas strings para o dataframe
df.dtypes

In [ ]:
#Nós podemos, contudo, dizer ao pandas que queremos mudar o tipo para categóricos, usando a função 'astype()'.
df['Notas'].astype('category').head(10)

In [ ]:
#Estamos vendo agora que temos 11 categorias, e o pandas está ciente de quais categorias são. Mais interessante é que nossos
#dados não são apenas categóricos. Mas ordenado, isso é, um A+ > A-. Podemos dizer ao pandas como organizar a ordem criando
#criando os dados categóricos que é uma lista das ordem categóricas, e então colocando ordenamento como true, veja:

categories = pd.CategoricalDtype(categories=['D','D+','C-','C','C+','B-','B','B+','A-','A','A+'],ordered = True)
# E passamos ela para a função 'astype()'
grades = df['Notas'].astype(categories)
grades.head()

In [ ]:
#Agora vemos que agora o pandas só não está ciente das onze categorias, como também sabe a ordem dessas categorias. Agora,
#O que podemos fazer com isso? Como há uma ordem, podemos usar para fazer comparações booleanas. Por exemplo, se quisermos
#comparar, as notas a uma nota C, que a comparação lexical não retorna exatamente o que esperavamos.

df[df['Notas'] > 'C'] #df não está organizado hierarquicamente, como o dataframe 'grades'

In [ ]:
#C+ é maior que C, mas D não. Contudo, se transmitirmos pelo dataframe que tem um tipo ordenado de categorias.

grades[grades > 'C']

In [ ]:
#Agora vemos que o operador funcionou como esperavamos. Nós podemos usar certos operações matemáticas, como maximo, minimos
#em nossos dados ordinários.

In [ ]:
#As vezes é útil representar dados categóricos como True ou False conforme aplicados a categorias. ISso é muito comum na extração
#de caracteristicas (features), que é um tópico sobre a mineração de dados. Variaveis com valores booleanos são geralmente
#chamadas de variaveis ficticias. E pandas tem função embutida chamada 'get_dummies' que converte valores de uma coluna
#em multiplas colunas de zeros e um, indicando presença de variaveis ficticias (dummy variable). Raramente usada, mas quando
#usada é muito útil.

In [ ]:
#Tem mais uma operação baseada em escalas que é interessante dizer. Que é converter algo de escala de proporção, ou de intervalo
#Para dados categoricos. Isso parece contra intuitivo pois estamos perdendo informação dos valores. Mas é bem usado, por exemplo.
#Se estivermos verificando a frequencia das categorias, isso pode ser uma abordagem bem útil, e histogramas são geralmente
#feitos de valores com intervalo, ou proporção. E mais, se estivermos trabalhando com aprendizado de maquina (Machine learning)
#para classificar, é preciso estar usando dados categóricos, então reduzir a dminesão é util para aplicar determinada técnica.

#Pandas tem uma função chamada 'cut()' que leva como argumento alguma estrutura de array (Uma coluna de um dataframe, Séries),
#Isso também leva um número de armazenamento a ser usado, e todos esses armazenamentos são mantidos em espaços iguais.

#Vamos novamente usar os dados do census para o exemplo. Nós vimos que agrupar por estados, então agregar para obter uma lista
#da média das contagens por estado. Se posteriormente, aplicarmos 'cut' para fazer isso, digamos, uns 10 armazenamentos (bins)
#podemos ver os estados listados como categóricos usando a média das contagens do estado

import numpy as np
import pandas as pd

df = pd.read_csv('census.csv')

#agora que trouxemos o dataset, vamos reduzir aos dados do país.
df = df[df['SUMLEV'] == 50]
#E agora para agrupar.
df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)

df.head()

In [ ]:
#agora queremos criar 'bins' em cada. podemos usar 'cut()', então:
pd.cut(df,10)

In [ ]:
#Aqui podemos observar que os estados do Alaska e Alabama caem na mesma categoria, enquanto california e Columbia caem em 
#categorias diferentes

#'Cutting' é apenas uma maneira de criar categorias em nossos dados, e existem outros métodos. POr exemplo.
#'CUt()' nos fornece intervalos de dados, aonde o espaço em cada categoria é do mesmo tamanho. Mas as vezes queremos
#formar categorias baseado na frequencia, queremos que o número de itens em cada 'bin' seja o mesmo, ao invés do espaço entre
#elas. Isso depende do formato dos seus dados e o que você pretende fazer com ele.

# Mesa pivô (Pivot table)

Uma mesa pivô é uma maneira de sintetizar os dados de um DataFrame para um próposito particular. Isso faz um pesado uso da função 'agg()'. Uma mesa pivô é por si mesma um DataFrame,   aonde as linhas representam um variavel de interesse, e as colunas outra, e as celulas outros valores agregados. Uma mesa pivô também tende a incluir valores marginais, que são as somas de cada coluna e linha. Isso nos permite ver a relação entre duas variaveis com mais clareza.

In [ ]:
#Vamos fazer uma mesa pivô nos pandas
import pandas as pd
import numpy as np

In [ ]:
# Aqui vamos usar os dados chamado 'Times Higher Education World University Ranking dataset', que é um dos mais influentes
#rankings das universidades. Vamos importa-lo

df = pd.read_csv('cwurData.csv')
df.head()

In [ ]:
#Aqui podemos ver cada instituição, seu rank nacional, qualidade de educação, outras métricas, e sua nota geral.
#Vamos dizer que queremos criar uma nova coluna chamada 'Nivel do rank', aonde instituições em posição de '1-100', aonde
#são caracterizados como 'Tier 1', de '101 - 200' como 'tier 2' e por ai adiante. 

#Agora já sabemos como fazer isso, então vamos:
def create_category(ranking):
  #Desde que o rank é apenas um inteiro. 
  if (ranking >= 1) & (ranking < 100):
    return ('Tier 1')
  elif (ranking >= 101) & (ranking <= 200):
    return ('Tier 2')
  elif (ranking >= 201) & (ranking <= 300):
    return ('Tier 3')
  return ('Low Tier (Bronze)')

#Agora podemos aplicar essa função essa coluna de dados para criar uma nova série.
df['rank_level'] = df['world_rank'].apply(lambda x: create_category(x))
df['rank_level'].head()

In [ ]:
# A mesa de pivô nos permite colocar como eixo uma dessas colunas, e comparar com outra coluna com os indices das linhas.
#Vamos dizer que queremos comparar o level do rank versus o país das universidades e nós queremos comparar os termos da 
#qualificação total.

#Para fazer isso, vamos dizer ao pandas que queremos os valores da qualificação,e o indice para ser os países e as colunas
#para ser rank levels. Então nós especificamos isso a função de agregação, e aqui nós iremos usar a média do numpy para ter as
#qualificação média das universidades nesse país.
df.pivot_table(values = 'score', index = 'country', columns = 'rank_level', aggfunc = [np.mean]).head()

In [ ]:
#Aqui vemos que há uma organização hierarquica no dataframe onde o indice, ou linhas, são por países e as colunas possuem
#dois level. Perceba também que temos alguns valores NaNs, por exemplo, a argentina indica que apenas tem universidades de 
#tier mais baixo.

#As meses de pivo não são limitadas a uma única função, que você pode querer aplicar, podemos passar um parametro chamado
#'aggfunc()', que é uma lista de diferentes funções a se aplicar. E o pandas irá te fornecer o resultado usando nomes de 
#colunas hierarquicamente

df.pivot_table(values = 'score', index = 'country', columns = 'rank_level', aggfunc=[np.mean, np.max]).head()

In [ ]:
#Então agora vemos que temos os dois, a média e o max. Como mencionado anteriormente, também podemos sintetizar os valores
# dentro de uma coluna de alto nivel. Por exemplo, queremos ver uma média geral do país para a média e queremos ver o máximo do
# máximo, podemos indicar ao pandas que queremos nos forneça valores marginais 
df.pivot_table(values='score', index  = 'country', columns = 'rank_level', aggfunc=[np.mean, np.max],
               margins = True).head()

In [ ]:
#Uma mesa pivot é apenas um dataframe multinivel, e podemos acessar séries, ou celulas no dataframe de maneira similar
#que fazemos com dataframes comuns

#Vamos criar um novo dataframe do nosso exemplo anterior
new_df = df.pivot_table(values = 'score', index='country', columns = 'rank_level',aggfunc=[np.mean, np.max],
                        margins = True)

print(new_df.index)
print(new_df.columns)

In [ ]:
#Podemos ver que as colunas são hierarquicas. Os indices das colunas de alto nivel possuem duas categorias: Média e max
#e as colunas de baixo nivel possuem quatro categorias, que são os ranks que definimos inicialmente. Como eu questiono o dataframe
#Se quisermos obter a média das notas das universidades de tier 1 em cada país? Apenas precisamos fazer duas projeções 
#de dataframes, o primeiro para média e o segundo para o Tier 1 das universidades.

new_df['mean']['Tier 1'].head()

In [ ]:
#Podemos ver que a saída é um objeto do tipo série, que pode ser confirmado por imprimir o tipo. Apenas lembrando que quando
#projetamos uma coluna pra fora do dataframe, nos é retornado uma série.

type(new_df['mean']['Tier 1'])

In [ ]:
#E se quisermos encontrar o país que possui a maior média das avaliações de universidades tier 1? Podemos usar idxmax() função
new_df['mean']['Tier 1'].idxmax()

In [ ]:
#A função idxmax() não é especial para mesas pivos, mas sim uma função embutida nos objetos do tipo Séries.
#Como não temos tempo para ir atraves de todas as funções e metodos existentes em pandas e numpy, encorajo você 
#para a explorar o API das bibliotecas e aprender mais independentemente.

In [ ]:
#Se quisermos atingir um diferente formato para nossa mesa pivo, podemos usar as funções 'stack' e 'unstack'.
#'Stacking' é girar a coluna mais baixa para a superior, e 'Unstacking' é exatamente a operação contrária

#Antes de ir para um exemplo, vamos relembrar nosso dataframe
new_df.head()

In [ ]:
#agora vamos tentar usar a função stack e transformar as colunas inferiores para as linhas superiores
new_df = new_df.stack()
new_df.head()

In [ ]:
#Bem agora vamos fazer um 'unstack' para ver o que sai
new_df = new_df.unstack()
new_df.head()

In [ ]:
#Vimos que ele retornou ao dataframe original, então podemos nos perguntar, o que acontece se fizermos 'unstack' duas vezes
#seguidas? 
new_df.unstack().unstack().head()

# Funcionalidades de Data/Horário

Agora vamos dar uma olhada nas funcionalidades de Data e horário do pandas. O pandas torna a manipulação bem flexivel e nos permite conduzir analises como analises de séries de tempos.

In [ ]:
import numpy as np
import pandas as pd

## Timestamp (Carimbo)

In [ ]:
#Pandas possui classes principais relacionadas ao tempo, 'Timestamp', 'DatetimeIndex', 'Period', 'PeriodIndex'. Primeiro,
#Vamos olhar para o timestamp. Isso representa um simples carimbo e associa os valores com pontos no tempo.

#Por exemplo, vamos criar uma string da data e horario de hoje, e isso será nosso timestamp, timestamp é intercalável com
#as funções de data/hora do python

pd.Timestamp('11/3/2021  9:46AM')

In [ ]:
#Também podemos criar um timestamp passando multiplos parametros, como ano, mês, dia, hora, minuto, separadamente
pd.Timestamp(2021, 11, 3, 9, 49)

In [ ]:
#Timestamp também alguns atributos bem uteis, como isoweekday(), que retornará o dia da semana
#Aonde 1 representa Segunda-feira, e 7 representa domingo

pd.Timestamp(2021,3,11,9,56).isoweekday()

In [ ]:
#Também podemos extrair o dia, mês, ano etc, de um timestamp
pd.Timestamp(2021, 3, 11, 9, 58).month

##Periodo

In [ ]:
#Supomos que não estamos interessanto em um ponto especifico no tempo, ao invés disso queremos um periodo de tempo. 
#Isso é onde a classe periodo entra na história. Periodo é um intervalo no tempo, como um dia especifico, ou um mês
pd.Period('3/2021')

In [ ]:
#Você pode perceber que foi indicado a dimensão do nosso intervalo, que no caso veio na saída um 'M' de 'month  , 
pd.Period('3/11/2021') #Lembrando que tem que organizar as horas pelo modelo americano

In [ ]:
#o periodo é todo o intervalo de tempo que especificarmos, e aritmética do periodo é intuitiva, por exemplo, se quisermos 
#os proximos 5 meses da data de hoje. 
pd.Period('3/2021') + 5

In [ ]:
#Ou se quisermos dois dias anteriores
pd.Period('3/11/2021') -2

##DatetimeIndex and PeriodIndex

In [ ]:
#O indice de um 'timestamp'(Carimbo de tempo) é um 'DatetimeIndex'. Vamos olhar um rápido exemplo. Primeiro, vamos criar
# nosso exemplo, vamos criar uma série com os 3 primeiros dias de um mês. Quando olharmos para a série, cada 'Timestamp' é
#o indice e tem um valor associada a ele, nesse caso, a,b,c.

t1 = pd.Series(list('abc'), [pd.Timestamp('2021/9/1'), pd.Timestamp('2021/9/2'),
                             pd.Timestamp('2021/9/3')])
t1

In [ ]:
#Agora vamos checar  o tipo do indice da nossa série
type(t1.index)

In [ ]:
#Similarmente podemos criar um indice baseado no periodo
t2 = pd.Series(list('def'), [pd.Period('2021-09'), pd.Period('2021-10'), pd.Period('2021-11')])
t2  

In [ ]:
#De forma similar, podemos checar o tipo do indice
type(t2.index)

##Convertendo to Data/Horario

In [ ]:
#Agora vamos dar uma olhada em como converter para data/hora. Suponha que temos uma lista de dados em strings e queremos criar
#um novo dataframe

#Vamos tentar diferentes formatos de data/horas.
d1 = [' 2 June 2013','Aug 29, 2014','2015-06-26','7/12/16']
#E alguns dados aleatórios
ts3 = pd.DataFrame(np.random.randint(10, 100, (4,2)), index = d1,
                   columns = list('ab'))
ts3

In [ ]:
#Usando pandas 'to_datetime', pandas irá tentar converter estes para data/hora e coloca-los em formato padrão

ts3.index = pd.to_datetime(ts3.index)
ts3

In [ ]:
#A função datetime também tem opções para mudar a ordem da data. Por exemplo, podemos passar o argumento 'dayfirst = True'
# para colocar o dia em primeiro e ficar no modelo europeu, e evitar o modelo americano.

pd.to_datetime("4.7.12", dayfirst=True)

##Timedelta

In [ ]:
# Timedeltas são diferenças de tempo, Isso não é o mesmo que periodo, mas conceitualmente similar. Por exemplo, se nós quisermos
# a difença entre 3/9 e 1/9, teremos um timeDelta(Intervalo de tempo) de 2 dias.

pd.Timestamp('9/3/2021') - pd.Timestamp('9/1/2021')

In [ ]:
#Nós também podemos fazer algo como encontrar que data e hora é 12dias e 3horas no passado de 2/9 as 8:10Am
pd.Timestamp('9/2/2021 8:10AM') + pd.Timedelta('12D 3h')

##Offset

In [ ]:
#Offset é semelhante ao timedelta, mas isso segue algumas regras do calendário. Offset permite flexibilidade em termos
#dos tipos de intervalos. além da hora, dia, semana, mês, etc. Isso também tem dia de negócios, fim de mês, quase inicio de mês

#Vamos criar um timestamp pra ver como é isso
pd.Timestamp('3/11/2021').weekday()

In [ ]:
#Agora podemos adicionar o timestamp com a semana a frente
pd.Timestamp('3/11/2021') +pd.offsets.Week() #Retornou o valor de uma semana a frente

In [ ]:
#Agora vamos tentar fazer isso com o fim do mês. então podemos ter o ultimo dia de março
pd.Timestamp('3/11/2021') + pd.offsets.MonthEnd() #Retornou o ultimo dia do mês

##Trabalhando com dados em um dataframe

In [ ]:
#Agora, vamos dar olhada em alguns truques ao se trabalhar com datas em um dataframe. Suponha que nós queremos olhar nove medidas
#coletadas quinzenalmente, todo domingo, iniciando em Outubro de 2016. Usando 'data_range', nós podemos criar esse DatetimeIndex
#em 'data_range' nós temos que especificar a data de inicio, ou a final, se não especificado, por padrão será a de inicio
#Então nós temos que especificar o número de periodos, e a frequencia. Aqui, nós colocaremos '2W-SUN', Que significa quinzenalmente
#(Duas semanas) iniciando no domingo

dates = pd.date_range('10-1-2016', periods = 9, freq = '2W-SUN')
dates

In [ ]:
#Existem muitas frequencias que podemos especificar, por exemplo, dia de trabalho (Business day)
pd.date_range('04-1-2016',periods = 12, freq = 'B')

In [ ]:
# Ou podemos fazer trimestralmente(Quartely), com o trimestre inicando em Junho
pd.date_range('04-01-2016', periods= 12, freq = 'QS-JUN')

In [ ]:
#Agora vamos voltar ao nosso exemplo de semanalmente inicando no domingo, e criar um dataframe usando esses dados e alguns dados
#aleatórios
dates = pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
df = pd.DataFrame({'count 1' : 100 + np.random.randint(-5, 10, 9).cumsum(),
                   'Count 2' : 120 + np.random.randint(-5, 10, 9)}, index = dates)
df

In [ ]:
#Primeiro, podemos checar que dia da semana a data especifica é (NO caso aqui, esperamos que seja todos domingos), Vamos ver se
#bate com a frenquencia que defininimos
df.index.weekday_name #Não sei o porque, mas isso não está funcionando

In [ ]:
#Podemos também usar diff() para encontrar a diferença entre o valor de cada data;
df.diff()

In [ ]:
#Suponha que queremos saber a média da contagem de cada mês, podemos fazer isso usando 'resample'. Converter de uma alta
#Para uma baixa frenquencia é chamado de 'downsampling' (Vamos falar disso em um momento)
df.resample('M').mean()

In [ ]:
#Agora vamos falar sobre datetime indexação e slicing, que é um maravilhoso recurso do pandas.
#Por exemplo, nós podemos usara indexação de uma string parcial para encontrar valores de um ano particular.
df['2017'] 

In [ ]:
#Ou podemos fazer isso para um mês especifico
df['2016-12']

In [ ]:
#Ou podemos mesmo fazer um slice do alcance dos dados. Por exemplo, aqui nós apenas queremos os valores de dezembro de 2016
#A frente.

df['2016-12' :]

In [ ]:
#Nós iremos falar de resampling em outra aula. E isso talvez seja amis claro de com usar isso. De novo, se tiver que lidar
#um montante de dados de data/hora, essa aula se tornará importante para voltar, revisar e entender. E também a funcionalidade
#do pandas em respeito a data/hora é bem fenomenal e a documentação descreve isso em mais detalhes
# Isso pode ser encontrado no link a seguir:
#https://pandas.pydata.org/docs/user_guide/timeseries.html